In [9]:
!export AWS_PROFILE=LWI

In [1]:
import os
os.environ['AWS_PROFILE'] = 'LWI'

In [2]:
import boto3
import json
from botocore.client import Config
from datetime import datetime, timezone


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from utils import S3ObjectMonitor
from utils.vector_pipeline import AddData
from utils import database_utils as db

In [27]:
connection,tables = AddData._load_config('credentials.yaml')

TypeError: _load_config() missing 1 required positional argument: 'config_file'

In [5]:
_MODE = "DEV"
_PROJECT = "LWI"
_DEFAULT_BUCKETS = {
    "PROD":[*[f"lwi-region{x}" for x in range(1, 8)]],
    "DEV":["lwi-common"]
}
_DEFAULT_PATHS = {
    "PROD":"deliverables/consequence_modeling_results/goconsequence_results_shp/",
    "DEV":"staging/wsel_test/"
}
_BUCKETS = _DEFAULT_BUCKETS[_MODE]
_PATH = _DEFAULT_PATHS[_MODE]
_STATE_FILE = "lwi_buckets_state.json"
_LAST_RUN = "lwi_last_run.json"
_FILE_TYPE = "shp"

In [6]:
# Create an instance of the S3ObjectMonitor class
monitor = S3ObjectMonitor(_PROJECT, _BUCKETS, _PATH, _STATE_FILE, _LAST_RUN,_FILE_TYPE)

# Monitor objects and perform the comparison
modified_objects = monitor.monitor_objects()
print(modified_objects)

{'added': [{'Bucket': 'lwi-common', 'Key': 'staging/wsel_test/AEP_0.5_wse_si_1_1.shp', 'LastModified': '2023-09-14T21:17:58+00:00'}], 'removed': [], 'updated': []}


In [63]:
results = []
for added in modified_objects["added"]:
    print(f"Added: {added}")
    results.append(AddData(path=added))

Added: {'Bucket': 'lwi-common', 'Key': 'staging/wsel_test/AEP_0.5_wse_si_1_1.shp', 'LastModified': '2023-09-14T21:17:58+00:00'}
Loading s3://lwi-common/staging/wsel_test/AEP_0.5_wse_si_1_1.shp


In [31]:
results[0].tables[0]['name']

'result'

In [66]:
processed_data

,gdb_geomattr_data,shape,storm_id,fd_id,x,y,depth,damage_cat,occupancy,structure,...,total_damage,pop2amu65,pop2amo65,pop2pmu65,pop2pmo65,path_aws,s_dam_per,c_dam_per,occupancy_str,damage_cat_str
0,NaN,POINT (-10046962.753 3559191.735),3,1932,-90.253402,30.431927,10.298351,Res,RES1-1SNB,157865.639285,...,215834.423182,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.661086,0.351816,Single Family 1 Story no basement,Residential
1,NaN,POINT (-10048128.157 3559130.022),3,1974,-90.263871,30.431449,11.573705,Res,RES1-1SNB,211566.803651,...,288727.774497,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.696346,0.368066,Single Family 1 Story no basement,Residential
2,NaN,POINT (-10048369.497 3559015.894),3,1979,-90.266039,30.430565,7.451999,Res,RES1-1SNB,281364.978106,...,370677.839009,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.578765,0.311382,Single Family 1 Story no basement,Residential
3,NaN,POINT (-10051230.074 3562040.787),3,2027,-90.291736,30.453992,17.998337,Res,RES1-1SNB,143656.289388,...,185920.761061,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.802160,0.400000,Single Family 1 Story no basement,Residential
4,NaN,POINT (-10051879.735 3562150.424),3,2044,-90.297572,30.454841,14.367069,Res,RES1-1SNB,189858.254158,...,257884.619847,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.755928,0.392535,Single Family 1 Story no basement,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222370,NaN,POINT (-10141244.573 3553341.094),3,2180972,-91.100350,30.386600,18.569178,Res,RES1-1SNB,139240.743134,...,186862.103134,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222371,NaN,POINT (-10142887.649 3552293.282),3,2180974,-91.115110,30.378480,28.901396,Res,RES1-1SNB,183683.764754,...,246505.004754,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222372,NaN,POINT (-10145708.485 3556574.119),3,2180977,-91.140450,30.411650,28.894518,Res,RES1-1SNB,151003.413290,...,195163.013290,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222373,NaN,POINT (-10145935.576 3555696.410),3,2180978,-91.142490,30.404850,26.008797,Res,RES1-1SNB,337809.299525,...,453342.579525,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential


In [65]:
from sqlalchemy import create_engine
processed_data = results[0].process_data()
processed_data.to_postgis("result", engine) 

In [70]:
processed_data.to_postgis("result", engine,if_exists='append',schema='owner')  

In [47]:
new_data.set_geometry('shape',inplace=True)
new_data.to_crs(3857,inplace=True)

In [49]:
new_data

,gdb_geomattr_data,shape,storm_id,fd_id,x,y,depth,damage_cat,occupancy,structure,...,total_damage,pop2amu65,pop2amo65,pop2pmu65,pop2pmo65,path_aws,s_dam_per,c_dam_per,occupancy_str,damage_cat_str
0,NaN,POINT (-10046962.753 3559191.735),3,1932,-90.253402,30.431927,10.298351,Res,RES1-1SNB,157865.639285,...,215834.423182,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.661086,0.351816,Single Family 1 Story no basement,Residential
1,NaN,POINT (-10048128.157 3559130.022),3,1974,-90.263871,30.431449,11.573705,Res,RES1-1SNB,211566.803651,...,288727.774497,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.696346,0.368066,Single Family 1 Story no basement,Residential
2,NaN,POINT (-10048369.497 3559015.894),3,1979,-90.266039,30.430565,7.451999,Res,RES1-1SNB,281364.978106,...,370677.839009,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.578765,0.311382,Single Family 1 Story no basement,Residential
3,NaN,POINT (-10051230.074 3562040.787),3,2027,-90.291736,30.453992,17.998337,Res,RES1-1SNB,143656.289388,...,185920.761061,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.802160,0.400000,Single Family 1 Story no basement,Residential
4,NaN,POINT (-10051879.735 3562150.424),3,2044,-90.297572,30.454841,14.367069,Res,RES1-1SNB,189858.254158,...,257884.619847,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.755928,0.392535,Single Family 1 Story no basement,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222370,NaN,POINT (-10141244.573 3553341.094),3,2180972,-91.100350,30.386600,18.569178,Res,RES1-1SNB,139240.743134,...,186862.103134,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222371,NaN,POINT (-10142887.649 3552293.282),3,2180974,-91.115110,30.378480,28.901396,Res,RES1-1SNB,183683.764754,...,246505.004754,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222372,NaN,POINT (-10145708.485 3556574.119),3,2180977,-91.140450,30.411650,28.894518,Res,RES1-1SNB,151003.413290,...,195163.013290,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222373,NaN,POINT (-10145935.576 3555696.410),3,2180978,-91.142490,30.404850,26.008797,Res,RES1-1SNB,337809.299525,...,453342.579525,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential


In [57]:
new_data['shape'] = new_data['shape'].apply(lambda x: f'ST_GeomFromText("{x}", 3857)')

/tmp/ipykernel_572066/2033574131.py:1: UserWarning: Geometry column does not contain geometry.
  new_data['shape'] = new_data['shape'].apply(lambda x: f'ST_GeomFromText("{x}", 3857)')


In [62]:
new_data

,gdb_geomattr_data,shape,storm_id,fd_id,x,y,depth,damage_cat,occupancy,structure,...,total_damage,pop2amu65,pop2amo65,pop2pmu65,pop2pmo65,path_aws,s_dam_per,c_dam_per,occupancy_str,damage_cat_str
0,NaN,"ST_GeomFromText(""POINT (-10046962.753000619 35...",3,1932,-90.253402,30.431927,10.298351,Res,RES1-1SNB,157865.639285,...,215834.423182,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.661086,0.351816,Single Family 1 Story no basement,Residential
1,NaN,"ST_GeomFromText(""POINT (-10048128.156749733 35...",3,1974,-90.263871,30.431449,11.573705,Res,RES1-1SNB,211566.803651,...,288727.774497,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.696346,0.368066,Single Family 1 Story no basement,Residential
2,NaN,"ST_GeomFromText(""POINT (-10048369.497405773 35...",3,1979,-90.266039,30.430565,7.451999,Res,RES1-1SNB,281364.978106,...,370677.839009,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.578765,0.311382,Single Family 1 Story no basement,Residential
3,NaN,"ST_GeomFromText(""POINT (-10051230.074360687 35...",3,2027,-90.291736,30.453992,17.998337,Res,RES1-1SNB,143656.289388,...,185920.761061,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.802160,0.400000,Single Family 1 Story no basement,Residential
4,NaN,"ST_GeomFromText(""POINT (-10051879.734908957 35...",3,2044,-90.297572,30.454841,14.367069,Res,RES1-1SNB,189858.254158,...,257884.619847,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.755928,0.392535,Single Family 1 Story no basement,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222370,NaN,"ST_GeomFromText(""POINT (-10141244.573089 35533...",3,2180972,-91.100350,30.386600,18.569178,Res,RES1-1SNB,139240.743134,...,186862.103134,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222371,NaN,"ST_GeomFromText(""POINT (-10142887.64877311 355...",3,2180974,-91.115110,30.378480,28.901396,Res,RES1-1SNB,183683.764754,...,246505.004754,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222372,NaN,"ST_GeomFromText(""POINT (-10145708.48466981 355...",3,2180977,-91.140450,30.411650,28.894518,Res,RES1-1SNB,151003.413290,...,195163.013290,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222373,NaN,"ST_GeomFromText(""POINT (-10145935.576431027 35...",3,2180978,-91.142490,30.404850,26.008797,Res,RES1-1SNB,337809.299525,...,453342.579525,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential


In [60]:
db.copy_from_stringio(results[0].connection,new_data,results[0].tables[0]['name'])

Error: extra data after last expected column
CONTEXT:  COPY result, line 1: ","ST_GeomFromText(""POINT (-10046962.753000619 3559191.7350791856)"", 3857)",3,1932,-90.253402,30.43..."



1

In [40]:
processed_data = results[0].process_data()
#results[0].save_data(processed_data)
columns = [
            'gdb_geomattr_data',
            'shape',
            'storm_id',
            'fd_id',
            'x',
            'y',
            'depth',
            ##'found_ht',
            ##'depth_above_ff',
            'damage_cat',
            'occupancy',
            'structure',
            'content_da',
            'total_damage',
            'pop2amu65',
            'pop2amo65',
            'pop2pmu65',
            'pop2pmo65',
            'path_aws',
            's_dam_per',
            'c_dam_per',
            'occupancy_str',
            'damage_cat_str']
new_data = processed_data[columns]
new_data

,gdb_geomattr_data,shape,storm_id,fd_id,x,y,depth,damage_cat,occupancy,structure,...,total_damage,pop2amu65,pop2amo65,pop2pmu65,pop2pmo65,path_aws,s_dam_per,c_dam_per,occupancy_str,damage_cat_str
0,NaN,POINT (-90.25340 30.43193),3,1932,-90.253402,30.431927,10.298351,Res,RES1-1SNB,157865.639285,...,215834.423182,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.661086,0.351816,Single Family 1 Story no basement,Residential
1,NaN,POINT (-90.26387 30.43145),3,1974,-90.263871,30.431449,11.573705,Res,RES1-1SNB,211566.803651,...,288727.774497,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.696346,0.368066,Single Family 1 Story no basement,Residential
2,NaN,POINT (-90.26604 30.43057),3,1979,-90.266039,30.430565,7.451999,Res,RES1-1SNB,281364.978106,...,370677.839009,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.578765,0.311382,Single Family 1 Story no basement,Residential
3,NaN,POINT (-90.29174 30.45399),3,2027,-90.291736,30.453992,17.998337,Res,RES1-1SNB,143656.289388,...,185920.761061,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.802160,0.400000,Single Family 1 Story no basement,Residential
4,NaN,POINT (-90.29757 30.45484),3,2044,-90.297572,30.454841,14.367069,Res,RES1-1SNB,189858.254158,...,257884.619847,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.755928,0.392535,Single Family 1 Story no basement,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222370,NaN,POINT (-91.10035 30.38660),3,2180972,-91.100350,30.386600,18.569178,Res,RES1-1SNB,139240.743134,...,186862.103134,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222371,NaN,POINT (-91.11511 30.37848),3,2180974,-91.115110,30.378480,28.901396,Res,RES1-1SNB,183683.764754,...,246505.004754,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222372,NaN,POINT (-91.14045 30.41165),3,2180977,-91.140450,30.411650,28.894518,Res,RES1-1SNB,151003.413290,...,195163.013290,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential
222373,NaN,POINT (-91.14249 30.40485),3,2180978,-91.142490,30.404850,26.008797,Res,RES1-1SNB,337809.299525,...,453342.579525,0,0,0,0,s3://lwi-common/staging/wsel_test/AEP_0.5_wse_...,0.807000,0.400000,Single Family 1 Story no basement,Residential


In [23]:
results[0].save_data(processed_data)

Before isnerting
Error: extra data after last expected column
CONTEXT:  COPY result, line 1: "POINT (-90.253402 30.431927),,2,1932,-90.253402,30.431927,10.298351287841797,Res,RES1-1SNB,157865.63..."



In [56]:
s31 = boto3.client('s3')

In [ ]:
# Initialize the S3 client
   s3 = boto3.client('s3')

 

   # Function to list objects in an S3 bucket
   def list_objects(bucket_name):
       response = s3.list_objects_v2(Bucket=bucket_name)
       return [obj['Key'] for obj in response.get('Contents', [])]

 

   # Function to save the current state to a file
   def save_state(state, filename):
       with open(filename, 'w') as f:
           json.dump(state, f)

 

   # Function to load the previous state from a file
   def load_state(filename):
       try:
           with open(filename, 'r') as f:
               return json.load(f)
       except FileNotFoundError:
           return []

 

   # S3 bucket name to monitor
   bucket_name = 'your-s3-bucket-name'

 

   # File to store the previous state
   state_file = 's3_state.json'

 

   # Load the previous state
   previous_state = load_state(state_file)

 

   # List objects in the S3 bucket
   current_state = list_objects(bucket_name)

 

   # Compare the current state to the previous state to detect changes
   added_objects = set(current_state) - set(previous_state)
   removed_objects = set(previous_state) - set(current_state)

 

   # Update the previous state with the current state
   save_state(current_state, state_file)

 

   # Print the changes
   print("Added Objects:", added_objects)
   print("Removed Objects:", removed_objects)